# Pre-procesamiento de datos

Todos los datos operados por un algoritmo de inteligencia artificial deben ser numéricos en una forma específica. Debido a que la mayoría de datos se encuentran en un formato diferente, que no puede ser utilizado en un algoritmo, estos deben ser convertidos a un formato adecuado. Esta tarea se conoce como `preprocessing`.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
# Leer los datos del archivo CSV original
df = pd.read_csv('telco.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [3]:
# Revisar la composición de datos en el DataFrame
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Objetivo

Aplicaremos pre-procesamiento de datos para realizar una predicción en la columna `MonthlyCharges`, intentando predecir los pagos mensuales de un cliente. Para ello, seleccionamos las columnas que puedan servir como características (`X`) y nuestra variable objetivo (`y`).

In [4]:
# Elegir las columnas a utilizar
columnas_importantes = [
    'gender',
    'Partner',
    'Dependents',
    'SeniorCitizen',
    'PhoneService',
    'MultipleLines',
    'InternetService',
    'MonthlyCharges'
]

# Nuevo DataFrame con las columnas seleccionadas
df_limpio = df[columnas_importantes]

In [5]:
# Revisamos una muestra de los datos de nuestro nuevo DataFrame
df_limpio.head()

,gender,Partner,Dependents,SeniorCitizen,PhoneService,MultipleLines,InternetService,MonthlyCharges
0,Female,Yes,No,0,No,No phone service,DSL,29.85
1,Male,No,No,0,Yes,No,DSL,56.95
2,Male,No,No,0,Yes,No,DSL,53.85
3,Male,No,No,0,No,No phone service,DSL,42.30
4,Female,No,No,0,Yes,No,Fiber optic,70.70


Como podemos observar, la mayoría de columnas con datos categóricos se encuentran en formato de texto. Si deseamos utilizar esos datos en un algoritmo de inteligencia artificial, necesitamos convertirlos en representaciones numéricas. Para ello, realizaremos una tarea de pre-procesamiento utilizando un codificador de tipo `LabelEncoder`.

In [6]:
# Crear el encoder para la columna gender
gender_encoder = LabelEncoder()

# "Aprender" los datos para el encoder.
# Este proceso reconoce las diferentes categorías en los datos.
# Para cada categoría, asignará un número iniciando desde 0
gender_encoder.fit(df_limpio['gender'])

# Podemos transformar los datos originales en sus representaciones numéricas
gender_encoder.fit_transform(df_limpio['gender'])

array([0, 1, 1, ..., 0, 1, 1])

In [7]:
# Si quisiéramos transofrmar los datos de forma inversa, es decir,
# obtener los datos originales a partir de sus representaciones,
# podemos utilizar el método invevrse_transform
gender_encoder.inverse_transform([0, 1, 1, 1, 0])

/home/chuik/venv/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['Female', 'Male', 'Male', 'Male', 'Female'], dtype=object)

In [8]:
# Podemos visualizar las clases encontradas por el encoder
gender_encoder.classes_

array(['Female', 'Male'], dtype=object)

## Una forma más eficiente

Ya que necesitamos un codificador para cada columna, crearemos una estructura de datos que permita almacenar un codificador distinto para cada columna.

In [9]:
# Empezamos revisando la de columnas del DataFrame
for columna in df_limpio.columns:
    print(columna)

gender
Partner
Dependents
SeniorCitizen
PhoneService
MultipleLines
InternetService
MonthlyCharges


Ya que no todas las columnas necesitan ser codificadas (`SeniorCitizen` ya se encuentra codificada y `MonthlyCharges` no es un dato de tipo categórico), incluiremos únicamente las columnas con datos categóricos que necesitan ser codificados.

In [10]:
# Creamos un nuevo DataFrame que tendrá los datos convertidos
df_final = pd.DataFrame()

# En un diccionario, almacenamos un codificador por cada columna
encoders = {
    'gender': LabelEncoder(),
    'Partner': LabelEncoder(),
    'Dependents': LabelEncoder(),
    'PhoneService': LabelEncoder(),
    'MultipleLines': LabelEncoder(),
    'InternetService': LabelEncoder(),
}

# Codificar cada columna y agregarla al nuevo DataFrame
for columna, encoder in encoders.items():
    encoder.fit(df_limpio[columna])
    df_final[columna] = encoder.fit_transform(df_limpio[columna])

df_final.head()

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService
0,0,1,0,0,1,0
1,1,0,0,1,0,0
2,1,0,0,1,0,0
3,1,0,0,0,1,0
4,0,0,0,1,0,1


In [11]:
# Revisión de todas las clases obtenidas por los codificadores
for column, encoder in encoders.items():
    print(column, encoder.classes_)
    print('=======')

gender ['Female' 'Male']
Partner ['No' 'Yes']
Dependents ['No' 'Yes']
PhoneService ['No' 'Yes']
MultipleLines ['No' 'No phone service' 'Yes']
InternetService ['DSL' 'Fiber optic' 'No']


In [12]:
# Agregamos las columnas que no transformamos al DataFrame final
df_final['SeniorCitizen'] = df_limpio['SeniorCitizen']
df_final['MonthlyCharges'] = df_limpio['MonthlyCharges']
df_final.head()

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,SeniorCitizen,MonthlyCharges
0,0,1,0,0,1,0,0,29.85
1,1,0,0,1,0,0,0,56.95
2,1,0,0,1,0,0,0,53.85
3,1,0,0,0,1,0,0,42.30
4,0,0,0,1,0,1,0,70.70


In [13]:
# Exportar los datos codificados a un nuevo archivo CSV
df_final.to_csv('datos_limpios.csv', index=None)